## NYC_Taxy_data_analysis

#### Queries
1. Add a column named as "Revenue" into dataframe which is the sum of the columns 'Fare_amount','Extra','MTA_tax','Improvement_surcharge','Tip_amount','Tolls_amount','Total_amount' 
2.  Increasing count of total passengers in New York City by area
3.  Realtime Average fare/total earning amount earned by 2 vendors
4. Moving Count of payments made by each payment mode
5.  Highest two gaining vendor's on a particular date with no of passenger and total distance by cab
6.  Most no of passenger between a route of two location.
7. Get top pickup locations with most passengers in last 5/10 seconds.

In [0]:
# File location and type
file_location = "/FileStore/tables/yellow_tripdata_2018_01__1_.parquet"
file_type = "parquet"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df.limit(5))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2018-01-01T00:21:05.000+0000,2018-01-01T00:24:23.000+0000,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null
1,2018-01-01T00:44:55.000+0000,2018-01-01T01:03:05.000+0000,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3,null,null
1,2018-01-01T00:08:26.000+0000,2018-01-01T00:14:21.000+0000,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.0,0.0,0.3,8.3,null,null
1,2018-01-01T00:20:22.000+0000,2018-01-01T00:52:51.000+0000,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.0,0.0,0.3,34.8,null,null
1,2018-01-01T00:09:18.000+0000,2018-01-01T00:27:06.000+0000,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55,null,null


In [0]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



##### 1. Add a column named as "Revenue" into dataframe which is the sum of the columns 'Fare_amount','Extra','MTA_tax','Improvement_surcharge','Tip_amount','Tolls_amount','Total_amount'

In [0]:
from pyspark.sql.functions import *
query_1_df=df.withColumn("Revenue",col('Fare_amount')+col('Extra')+col('MTA_tax')+col('Improvement_surcharge')+col('Tip_amount')+col('Tolls_amount')+col('Total_amount'))
display(query_1_df.limit(5))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Revenue
1,2018-01-01T00:21:05.000+0000,2018-01-01T00:24:23.000+0000,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null,11.6
1,2018-01-01T00:44:55.000+0000,2018-01-01T01:03:05.000+0000,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3,null,null,30.6
1,2018-01-01T00:08:26.000+0000,2018-01-01T00:14:21.000+0000,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.0,0.0,0.3,8.3,null,null,16.6
1,2018-01-01T00:20:22.000+0000,2018-01-01T00:52:51.000+0000,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.0,0.0,0.3,34.8,null,null,69.6
1,2018-01-01T00:09:18.000+0000,2018-01-01T00:27:06.000+0000,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55,null,null,33.1


##### 2. Increasing count of total passengers in New York City by area

In [0]:
query_2_df=(df.groupby("PULocationID", "DOLocationID").sum("passenger_count"))
display(query_2_df.limit(5))

PULocationID,DOLocationID,sum(passenger_count)
79,116,580
246,249,5073
234,144,7047
161,193,100
231,261,5812


##### 3. Realtime Average fare/total earning amount earned by 2 vendors

In [0]:
query_3_df=df.groupby("VendorID").sum('fare_amount')
display(query_3_df.limit(5))

VendorID,sum(fare_amount)
1,4.599966776999971E7
2,6.126991590999834E7


In [0]:
df \
    .where("VendorID is not null")\
    .groupBy("VendorID")\
    .agg(sum("fare_amount"))\
    .select("VendorID",col("sum(fare_amount)").alias("Total_of_Fare_Amount"))\
    .orderBy("VendorID")\
    .show()

+--------+--------------------+
|VendorID|Total_of_Fare_Amount|
+--------+--------------------+
|       1| 4.599966776999971E7|
|       2| 6.126991590999834E7|
+--------+--------------------+



##### 4. Moving Count of payments made by each payment mode

In [0]:
query_4_df=df.groupby("payment_type").agg(count("passenger_count"))
display(query_4_df.limit(5))

payment_type,count(passenger_count)
1,6106416
3,43204
2,2599215
4,11852


d
##### 6. Most no of passenger between a route of two location

In [0]:
query_6_df=(df.groupby("PULocationID","DOLocationID")
     .agg(sum("passenger_count")))
display(query_6_df.limit(5))

PULocationID,DOLocationID,sum(passenger_count)
79,116,580
246,249,5073
234,144,7047
161,193,100
231,261,5812
